In [ ]:
%load_ext autoreload

import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize

import pyOptLE as optle

In [ ]:
%autoreload

In [ ]:
RT = 1.
beta = 1/RT

n_knots = 20

##############################

generate = False

if generate:
    dt = 0.1
    D = 0.5
    beta = 1
    n = 50
    N = 1000
#     V = optle.harmonic_V
#     nablaV = optle.harmonic_nablaV
    V = optle.V_dblwell
    nablaV = optle.nablaV_dblwell
    q = optle.overdamped_Langevin(n, N, dt, beta, D, nablaV)
    f = np.zeros_like(q)
else:
    # format data from "real" simulation
    filename = 'data/'
    
    q, f = optle.load_traj(filename)

plt.plot([qi.shape[0] for qi in q])
plt.xlabel('Trajectory index')
plt.ylabel('N. of pts per trajectory')


In [ ]:
# Precompute displacements
deltaq = [qi[1:] - qi[:-1] for qi in q]
# deltaq = np.array(deltaq, dtype=object)

epsilon = 1e-10
# Add margin for floating-point precision
qmin = min([np.min(qi) for qi in q]) - epsilon
qmax = max([np.max(qi) for qi in q]) + epsilon

# import multiprocessing
# num_processes = multiprocessing.cpu_count()

knots = np.linspace(qmin, qmax, n_knots)

# Initial guess for the parameters: zero free energy gradient, constant D = 1
initial_params = np.concatenate((np.zeros(n_knots), np.zeros(n_knots) + 1.))


In [ ]:
%autoreload

In [ ]:
start_time = time.time()
print(optle.objective_order1_debiased(initial_params, knots, q, deltaq, dt, beta, f))
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
print(optle.objective_order1_debiased(initial_params, knots, q, deltaq, dt, beta, f))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:

def history(res):
    history.opt.append(res)
    if history.nsteps%100 == 0:
        print('Step', history.nsteps, np.square(res).sum())
    history.nsteps += 1

history.opt = list()
history.nsteps = 0

start_time = time.time()
# Minimize the objective function using L-BFGS-B algorithm
result = minimize(optle.objective_order1_debiased, initial_params,  args=(knots, q, deltaq, dt, beta, f), jac=True, method='L-BFGS-B', callback=history)
print("--- %s seconds ---" % (time.time() - start_time))

print(result.nit, result.message)

In [ ]:
optimized_params = result.x

# Extract the optimized knots and coefficients
optimized_knots = knots
optimized_G = optimized_params[:n_knots]
optimized_logD = optimized_params[n_knots:]

interp_factor = 10

# Fencepost theorem aplied forward and then backward
n_discr_points = interp_factor * (len(knots) - 1) + 1
x = np.linspace(qmin, qmax, n_discr_points)


%matplotlib inline
# Plot the original data and the optimized spline
import matplotlib.pyplot as plt

counts, bins = np.histogram(np.concatenate(q), bins=40)
# FE = -RT * np.log(counts)
# FE -= np.min(FE)
# plt.stairs(FE, bins, label='FE from histogram')
plt.stairs(-RT*(np.log(counts)-np.max(np.log(counts))), bins, label='\"Free energy\" from histogram')


# TODO reuse this for ABF
# Integral of linear interpolation

predicted = optle.piecewise_linear_int(x, knots, optimized_G)
plt.plot(x, predicted, label='Optimized free energy')
plt.xlabel('colvar q')
plt.ylabel('Free energy')

if generate:
    plt.plot(x, V(x) - np.min(V(x)), label='True potential')

# Ad hoc bc we know this is a double well
plt.plot(x, optle.V_dblwell(x) - np.min(optle.V_dblwell(x)), label='Double well')

plt.legend()

    
    
plt.figure()
plt.plot(x, np.exp(np.interp(x, knots, optimized_logD)), label='Optimized D')
if generate:
    plt.plot([qmin, qmax], [D, D], label='True D')
plt.xlabel('colvar q')
plt.ylabel('Diffusion coefficient')
plt.legend()

plot_gradient = False
if plot_gradient:
    plt.figure()
    predicted_nablaV = np.interp(x, knots, optimized_G)
    plt.scatter(knots, optimized_G, label='Optimized grad G')
    plt.plot(x, predicted_nablaV, label='Optimized gradient')
    if generate:
        plt.plot(x, np.interp(x, knots, nablaV(knots)), label='True gradient')
    plt.xlabel('colvar q')
    plt.ylabel('Diffusion coefficient')
    plt.legend()


In [ ]:
no_bias = [np.zeros_like(fi) for fi in f]


history.opt = list()
history.nsteps = 0

start_time = time.time()
# Minimize the objective function using L-BFGS-B algorithm
result = minimize(optle.objective, initial_params,  args=(q, deltaq, no_bias, knots), jac=True, method='L-BFGS-B', callback=history)
print("--- %s seconds ---" % (time.time() - start_time))

print(result.nit, result.message)

In [ ]:
optimized_params = result.x

# Extract the optimized knots and coefficients
optimized_knots = knots
optimized_G = optimized_params[:n_knots]
optimized_logD = optimized_params[n_knots:]

interp_factor = 10

# Fencepost theorem aplied forward and then backward
n_discr_points = interp_factor * (len(knots) - 1) + 1
x = np.linspace(qmin, qmax, n_discr_points)


%matplotlib inline
# Plot the original data and the optimized spline
import matplotlib.pyplot as plt

counts, bins = np.histogram(np.concatenate(q), bins=40)
# FE = -RT * np.log(counts)
# FE -= np.min(FE)
# plt.stairs(FE, bins, label='FE from histogram')
plt.stairs(-RT*(np.log(counts)-np.max(np.log(counts))), bins, label='\"Free energy\" from histogram')


# TODO reuse this for ABF
# Integral of linear interpolation

predicted = optle.piecewise_linear_int(x, knots, optimized_G)
plt.plot(x, predicted, label='Optimized free energy')
plt.xlabel('colvar q')
plt.ylabel('Free energy')
plt.legend()

if generate:
    plt.scatter(x, V(x), label='True potential')
    plt.legend()

plt.figure()
plt.plot(x, np.exp(np.interp(x, knots, optimized_logD)), label='Optimized D')
if generate:
    plt.plot([qmin, qmax], [D, D], label='True D')
plt.xlabel('colvar q')
plt.ylabel('Diffusion coefficient')
plt.legend()


plot_gradient = False
if plot_gradient:
    plt.figure()
    predicted_nablaV = np.interp(x, knots, optimized_G)
    plt.scatter(knots, optimized_G, label='Optimized grad G')
    plt.plot(x, predicted_nablaV, label='Optimized gradient')
    if generate:
        plt.plot(x, np.interp(x, knots, nablaV(knots)), label='True gradient')
    plt.xlabel('colvar q')
    plt.ylabel('Diffusion coefficient')
    plt.legend()
